# Цель: сгенерировать текст стилем автора, используя собственную или предобученную LLM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch as th

from google.colab import drive
from google.colab import files

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

from warnings import filterwarnings
filterwarnings('ignore')

sns.set_style('darkgrid')

### Для чтения текста

In [ ]:
!pip install Spire.Doc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 26.0 MB/s eta 0:00:00


### Очищаем .p65

In [ ]:
!find . -name "*.p65" -delete

### Считываем все файлы

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


# Чтение текстов с файлов

In [ ]:
from spire.doc import *
from spire.doc.common import *

In [ ]:
doc = Document()
doc.LoadFromFile('/content/drive/MyDrive/data/Grani/Glava2.rtf')
text = doc.GetText()

In [ ]:
text

'Evaluation Warning: The document was created with Spire.Doc for Python.\r\n\r\n Министерство образования Российской Федерации\r\n\r\n\r\nРОССИЙСКИЙ ГОСУДАРСТВЕННЫЙ УНИВЕРСИТЕТ НЕФТИ И ГАЗА им. И.М. ГУБКИНА\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nВ. И. БОЛЬШАКОВ\r\n\r\n\r\n\r\n\r\nДИНАМИКА КУЛЬТУРНО-ЦИВИЛИЗАЦИОННОГО ПРОЦЕССА\r\n\r\n\r\n\r\n\r\n\r\nУчебное пособие\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nМосква 2016\r\nУДК 316.334:37\r\nББК 50.5\r\n\r\n \r\nБольшаков В.И. Динамика культурно-цивилизационного процесса. Учебное пособие. М.: РГУ нефти и газа им. И.М. Губкина. 2009 г., 200 с.\r\n\r\n\r\n\r\nВ настоящем издании сделана попытка применения системно-функционального метода для анализа сравнительного исследования социально-политического процесса развития глобальных социально-политических систем в широкой исторической ретроспективе. Отмечается общее и особенное в социально-политических системах, присущее различным мировым цивилизациям. Обосновывается не

In [ ]:
!find . -name "*.doc"

./drive/MyDrive/data/Пособие-Динамика-2016.doc
./drive/MyDrive/data/Сакральность6 власти.doc
./drive/MyDrive/data/Для перевода Без правки_ЕА_Святая-6 Земля.doc
./drive/MyDrive/data/Вся книга-2.doc
./drive/MyDrive/data/Skobary/Skoba1Pravka.doc


In [ ]:
!find . -name "*.DOC"

./drive/MyDrive/data/По закону/S2-w.DOC
./drive/MyDrive/data/По закону/BIBL-S-w.DOC
./drive/MyDrive/data/По закону/S0-w.DOC
./drive/MyDrive/data/По закону/S1-w.DOC
./drive/MyDrive/data/По закону/S3-w.DOC
./drive/MyDrive/data/По закону/S4-w.DOC
./drive/MyDrive/data/По закону/S5-w.DOC
./drive/MyDrive/data/По закону/S6-w.DOC
./drive/MyDrive/data/По закону/S7-w.DOC
./drive/MyDrive/data/По закону/S8-w.DOC
./drive/MyDrive/data/По закону/S9-w.DOC
./drive/MyDrive/data/Skobary/GL-АВ-99.DOC


In [ ]:
!find . -name "*.docx"

./drive/MyDrive/data/Большаков-Галерея-текущий иариант.docx
./drive/MyDrive/data/Власть-9-редактору.docx
./drive/MyDrive/data/8а-4ВСЯ-Наследие предков.docx
./drive/MyDrive/data/Время собирать камни-Фрагмент.docx
./drive/MyDrive/data/ЕА_на верстку без правки.docx


In [ ]:
files_to_process = ['./drive/MyDrive/data/Пособие-Динамика-2016.doc',
               './drive/MyDrive/data/Сакральность6 власти.doc',
               './drive/MyDrive/data/Для перевода Без правки_ЕА_Святая-6 Земля.doc',
               './drive/MyDrive/data/Вся книга-2.doc'
               './drive/MyDrive/data/Skobary/Skoba1Pravka.doc',
               './drive/MyDrive/data/По закону/S2-w.DOC',
               './drive/MyDrive/data/По закону/BIBL-S-w.DOC',
               './drive/MyDrive/data/По закону/S0-w.DOC',
               './drive/MyDrive/data/По закону/S1-w.DOC',
               './drive/MyDrive/data/По закону/S3-w.DOC',
               './drive/MyDrive/data/По закону/S4-w.DOC',
               './drive/MyDrive/data/По закону/S5-w.DOC',
               './drive/MyDrive/data/По закону/S6-w.DOC',
               './drive/MyDrive/data/По закону/S7-w.DOC',
               './drive/MyDrive/data/По закону/S8-w.DOC',
               './drive/MyDrive/data/По закону/S9-w.DOC',
               './drive/MyDrive/data/Skobary/GL-АВ-99.DOC',
               './drive/MyDrive/data/Большаков-Галерея-текущий иариант.docx',
               './drive/MyDrive/data/Власть-9-редактору.docx',
               './drive/MyDrive/data/8а-4ВСЯ-Наследие предков.docx',
               './drive/MyDrive/data/Время собирать камни-Фрагмент.docx',
               './drive/MyDrive/data/ЕА_на верстку без правки.docx',
               './drive/MyDrive/data/Vozlubi/Glava1.rtf',
               './drive/MyDrive/data/Grani/Glava1.rtf',
               './drive/MyDrive/data/Grani/Glava2.rtf',
               './drive/MyDrive/data/Grani/Glava4.rtf',
               './drive/MyDrive/data/Grani/Glava3.rtf',
               './drive/MyDrive/data/Grani/Glava5.rtf',
               './drive/MyDrive/data/Grani/Pril.rtf',
               './drive/MyDrive/data/Grani/Glava6.rtf',
               './drive/MyDrive/data/Grani/epilog.rtf',
               './drive/MyDrive/data/Vozlubi/пролог.RTF',
               './drive/MyDrive/data/Vozlubi/5.RTF',
               './drive/MyDrive/data/Vozlubi/4.RTF',
               './drive/MyDrive/data/Vozlubi/3.RTF',
               './drive/MyDrive/data/Vozlubi/2.RTF',
               './drive/MyDrive/data/Vozlubi/эпилог.RTF',
               './drive/MyDrive/data/Grani/primech.RTF',
               './drive/MyDrive/data/Skobary/Skoba1Pravka.RTF'
]